In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Lambda,Dense
import tensorflow.keras.backend as K

2025-11-10 03:01:03.854819: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-10 03:01:04.493328: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-10 03:01:06.483583: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
# Textual Document 1
text = """The speed of transmission is an important point of difference between the two viruses. 
Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) 
and a shorter serial interval (the time between successive cases) than COVID-19 virus. 
The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. 
This means that influenza can spread faster than COVID-19. 
Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus 
before the appearance of symptoms – is a major driver of transmission for influenza. 
In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, 
at present, this does not appear to be a major driver of transmission. 
The reproductive number – the number of secondary infections generated from one infected individual – 
is understood to be between 2 and 2.5 for COVID-19 virus, higher than for influenza."""

# Load textual document from file
# with open("document1.txt", "r", encoding="utf-8") as f:
    # text = f.read()

In [3]:
# Tokenize text
tok=Tokenizer()
tok.fit_on_texts([text])
seq=tok.texts_to_sequences([text])[0]
vocab_size=len(tok.word_index)+1

word_index=tok.word_index
index_word={i: w for w, i in word_index.items()}

print("Vocabulary size:", vocab_size)
print("Sequence:", seq)

Vocabulary size: 92
Sequence: [1, 37, 2, 4, 5, 38, 39, 40, 2, 41, 12, 1, 42, 43, 6, 44, 11, 20, 45, 46, 47, 1, 21, 22, 48, 7, 23, 2, 24, 25, 11, 20, 13, 14, 1, 21, 12, 49, 50, 15, 8, 9, 3, 1, 13, 14, 10, 8, 9, 3, 5, 51, 7, 16, 17, 52, 18, 26, 10, 6, 3, 1, 13, 14, 5, 27, 18, 28, 53, 29, 6, 30, 54, 55, 15, 8, 9, 56, 4, 31, 1, 57, 27, 17, 18, 2, 58, 59, 60, 61, 62, 4, 63, 2, 1, 3, 64, 1, 23, 2, 24, 19, 5, 11, 32, 33, 2, 4, 10, 6, 31, 65, 26, 66, 34, 67, 29, 68, 34, 69, 70, 30, 71, 8, 9, 3, 72, 73, 74, 75, 7, 76, 77, 78, 79, 28, 80, 81, 82, 7, 16, 11, 32, 33, 2, 4, 1, 83, 35, 19, 1, 35, 2, 84, 85, 86, 22, 87, 88, 89, 19, 5, 90, 7, 16, 12, 36, 25, 36, 17, 10, 8, 9, 3, 91, 15, 10, 6]


In [4]:
# GENERATE TRAINING DATA (CBOW)

window=2
x,y=[],[]

for i in range(window,len(seq)-window):
    context=seq[i-window:i]+seq[i+1:i+window+1]
    x.append(context)
    y.append(seq[i])

x=np.array(x)
y=np.array(y)

print("\nContext samples:\n", x[:5])
print("\nTarget samples:\n", y[:5])


Context samples:
 [[ 1 37  4  5]
 [37  2  5 38]
 [ 2  4 38 39]
 [ 4  5 39 40]
 [ 5 38 40  2]]

Target samples:
 [ 2  4  5 38 39]


In [5]:
# TRAIN CBOW MODEL

model=Sequential([
    Embedding(vocab_size,8),
    Lambda(lambda x: K.mean(x,axis=1)),
    Dense(vocab_size,activation='softmax')
])

In [6]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam'
)
model.fit(
    x,y,
    epochs=200,verbose=0
)
model.summary()
print("Model train completed")

2025-11-10 03:01:13.915130: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 4, 8)           │           736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 92)             │           828 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,694 (18.34 KB)

 Trainable params: 1,564 (6.11 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,130 (12.23 KB)

Model train completed


In [7]:
embedding=model.layers[0].get_weights()[0]

print("\n Word Embedding:")
for word,idx in word_index.items():
    print(f"{word} -> {embedding[idx]}")


 Word Embedding:
the -> [-1.9365581   2.3502889   1.7676018   0.3083103  -0.24087842  1.3263885
 -0.0614363  -1.2402346 ]
of -> [1.30445   1.1739161 0.5173527 0.3794296 1.1269379 0.5639042 0.8557598
 1.308143 ]
virus -> [-1.1059805  -0.11717158  1.7076402  -0.91681355 -0.3065731  -0.9630749
  1.4318334   1.052894  ]
transmission -> [-0.40268007  0.34516805  1.355357   -0.1455054   1.2266114   0.363463
  0.43540257 -0.92077166]
is -> [ 0.11682857  1.4034642  -0.9419608  -0.2961414   0.5118464   0.01627076
  0.9897958   0.7943002 ]
influenza -> [-1.2063636   0.5848356  -0.5304847   1.2813439  -0.84113836  0.42399657
  0.576857   -0.1120891 ]
to -> [ 0.891862   -0.861094    0.7926654  -1.7180996   0.4996402   0.32990196
 -0.4097618  -0.26700717]
covid -> [-1.2368681   0.33805606  0.4745931   2.0006702  -1.4646004  -1.378688
  1.4354037   1.4119023 ]
19 -> [-1.2500076  -0.29772007  0.94124013 -0.07346696 -0.9874633   0.10950226
  1.3574537   1.9204438 ]
for -> [-1.5161089  -0.7671925   0.

In [8]:
def predict_missing(w1, w2, w3, w4):
    try:
        seq = np.array([word_index[w1], word_index[w2], word_index[w3], word_index[w4]]).reshape(1, 4)
    except KeyError as e:
        return f"Word not found in vocabulary: {e}"

    pred = model.predict(seq, verbose=0)
    pred_id = int(np.argmax(pred))
    return index_word[pred_id]

print("\nPredictions:")
print(predict_missing("the", "speed", "transmission", "is"))
print(predict_missing("the", "serial", "for", "virus"))


Predictions:
of
virus
